# Import usefull packages

In [1]:
import spotipy
import pandas as pd
import json
import pprint
import config
from itertools import *
from bs4 import BeautifulSoup as bs
import sys
import random
import requests
import pandas as pd
import pickle
from spotipy.oauth2 import SpotifyClientCredentials
sp= spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=config.client_id,client_secret=config.client_secret))

# Create our final df

 - for faster workflow without artist and song name
 - called big_df

In [2]:
big_df=pd.DataFrame(columns=["danceability","energy","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo","id","duration_ms"])

# Create function get_audio_features()

In [3]:
def get_features_df(username, playlist_id):
    global big_df
    list_of_audio_features=[]
    artist_name=[]
    song_name=[]
    
    results = sp.user_playlist_tracks(username,playlist_id,market="DE")
    
    tracks = results['items']
    
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
        
#getting features per song in  playlist 
    for value in range(0,len(tracks)):
        #print (songs[value]["track"]["id"])
        list_of_audio_features.append(sp.audio_features(tracks[value]["track"]["id"])[0])
    
#getting  artist name per song in playlist
    #for a_name in range(0,len(tracks)):
        #print (songs[a_name]["track"]["artists"][a_name]["name"])
        #artist_name.append(tracks[a_name]["track"]["artists"][0]["name"])  

#getting  song name per song in playlist   
    #for s_name in range(0,len(tracks)):
    #print (songs[s_name]["track"]["name"])
    #song_name.append(tracks[s_name]["track"]["name"])

#creating df with audio features and adding song name and artist     
    df=pd.DataFrame(list_of_audio_features)    
    df=df[["danceability","energy","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo","id","duration_ms"]]
    
    #df.insert(0,"song", song_name)
    #df.insert(1,"artist", artist_name)
   
    big_df =big_df.append(df)
    
    return big_df

## save function

In [4]:
with open("Model/get_features_df.pickle", "wb") as f:
    pickle.dump(get_features_df,f)

## Test with private playlist


In [5]:
get_features_df("familie.oeljeklaus","3zGCtRVyGrZ2PH6ipGnsNv")


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_15011/1552337488.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms
0,0.865,0.665,-6.289,0.073,0.0453,0.0,0.224,0.331,111.113,1ofhfV90EnYhEr7Un2fWiv,268827
1,0.801,0.91,-6.566,0.225,0.00239,0.000891,0.0737,0.805,87.277,5g9lS8deSIxItFBmZRC4vN,278387
2,0.644,0.846,-5.443,0.275,0.124,0.0,0.353,0.503,79.182,1qZqFtSINOOmjYH1UE9805,301600
3,0.802,0.787,-6.009,0.206,0.00127,0.000181,0.0763,0.806,90.189,7sKt5Y2zJpYw78iAxndB8y,296907
4,0.872,0.723,-5.968,0.247,0.0293,0.0,0.327,0.843,93.279,4VQNCzfZ3MdHEwwErNXpBo,307773
...,...,...,...,...,...,...,...,...,...,...,...
197,0.76,0.585,-7.307,0.161,0.0102,0.000468,0.308,0.455,91.179,6uWez0py1zc17F7hjp7jIt,311040
198,0.888,0.584,-13.308,0.112,0.0385,0.000003,0.0625,0.762,104.705,5vpDsiVqiZLRwKAzQO708k,334440
199,0.916,0.844,-3.967,0.236,0.0394,0.0,0.0778,0.586,95.19,0Z2J91b2iTGLVTZC4fKgxf,312627
200,0.711,0.693,-8.208,0.41,0.106,0.0,0.0821,0.906,177.978,0LfoVjNyrzkCLzarDAazCh,229305


# Webscraping for playlists

In [6]:
url = "https://everynoise.com/everynoise1d.cgi?root=alt%20z&scope=all"
response = requests.get(url)
response.status_code


200

In [7]:
soup = bs(response.content, "html.parser")

In [8]:
soup.select("table tr td a.note ")[0]["href"].rsplit(":")[-1]

'6TySxsLwRVYqEh7LZ6fyq8'

In [9]:
a = soup.select("table tr td .note")
playlist_list = [str(i['href']) for i in a]
id_list = [i.split(':')[3] for i in playlist_list]
id_list


['6TySxsLwRVYqEh7LZ6fyq8',
 '0OGpuF8rnCzlcjJLsdywny',
 '1BYospE2cLB28m3i84dcTV',
 '6gS3HhOiI17QNojjPuPzqc',
 '5nUyv9wyZKjdUW1rk3smTo',
 '7nHgmv7uyIA1KHj6qTttjH',
 '0lXSqruCjvX1e7hw2KGvKw',
 '41997GIfsUgMnsPLFiuLeS',
 '1aYiM4zLmBuFq0Fg6NQb6a',
 '10FCW9lj0NdeoYI5VVvVtY',
 '6TGOKSobwpEoOtx80edFyV',
 '29qGA62efPsgLp6kblUPSg',
 '67qIuPZRX12l7yMpxBQpxs',
 '2QYND559WzeB2urcTaOG0j',
 '6rZGKd5ndSUnGeJKuBhUej',
 '2XXreCGINlOqm8wEn7WszA',
 '0ShRnc2cyEyxkLbtZC4Yim',
 '5kdP7VQedYdDzhTvfovu0u',
 '4CfyHGpt8qPlLXlstcE2Wh',
 '1Rn01P3YPioy5QRQoo32Lv',
 '0yVmsZK4AtiNMpby2WqLZY',
 '7mBz3DA4Lt7Bsbtr5TxklP',
 '7AqOmLV0JV4a3GpIU1qxbL',
 '5yagbqWY6gDlqB893zjOmO',
 '1HGdDFrmuZ9cfLJwJ7Eb3s',
 '0FkLYgMF09lmzytexAraKv',
 '0hkXwJmcwu7hABTyspKI9b',
 '47bikoA9qfXDT5VVhzsjy9',
 '5z1A6hdw1MJrmZzDusE4Q6',
 '08RIk2wKqMNCSzEaZCcHhi',
 '6ZbHMh1injVPBZTXnDKnYp',
 '78IYTCUA99H1rwKXjTYSCO',
 '3ejnp6KkIXiZBWsCzujQ6K',
 '49KNl5r9iJkG8BYo8VlWxX',
 '2BeLGTqy54PhX1YT1jTts9',
 '5VnHdgsAhzWNkd29Rv9ErH',
 '3TSPcCQfHfv9ga1UTKeBTH',
 

# Create list with playlist

In [19]:
username_=["familie.oeljeklaus",
          "paulraensch",
          "thesoundsofspotify",
         "thesoundsofspotify",
         ]
playlist_id_=["2RHNBm388mdIGQyH3ZGkaY",
             "6CDTDuRKy2Ze2nmmYFAIBK",
             "69fEt9DN5r4JQATi52sRtq",
            "4L9GKyeqZOi5rHUoqDZd6z",
            ]


## This list contains around 6000 playlist from different genres 

In [11]:
len(id_list)

5957

## Take samples of the list each contains 250

In [14]:
chunk_size= 24
result=[id_list[i:i + chunk_size] for i in range(0, len(id_list), chunk_size)]
print(result)

[['6TySxsLwRVYqEh7LZ6fyq8', '0OGpuF8rnCzlcjJLsdywny', '1BYospE2cLB28m3i84dcTV', '6gS3HhOiI17QNojjPuPzqc', '5nUyv9wyZKjdUW1rk3smTo', '7nHgmv7uyIA1KHj6qTttjH', '0lXSqruCjvX1e7hw2KGvKw', '41997GIfsUgMnsPLFiuLeS', '1aYiM4zLmBuFq0Fg6NQb6a', '10FCW9lj0NdeoYI5VVvVtY', '6TGOKSobwpEoOtx80edFyV', '29qGA62efPsgLp6kblUPSg', '67qIuPZRX12l7yMpxBQpxs', '2QYND559WzeB2urcTaOG0j', '6rZGKd5ndSUnGeJKuBhUej', '2XXreCGINlOqm8wEn7WszA', '0ShRnc2cyEyxkLbtZC4Yim', '5kdP7VQedYdDzhTvfovu0u', '4CfyHGpt8qPlLXlstcE2Wh', '1Rn01P3YPioy5QRQoo32Lv', '0yVmsZK4AtiNMpby2WqLZY', '7mBz3DA4Lt7Bsbtr5TxklP', '7AqOmLV0JV4a3GpIU1qxbL', '5yagbqWY6gDlqB893zjOmO'], ['1HGdDFrmuZ9cfLJwJ7Eb3s', '0FkLYgMF09lmzytexAraKv', '0hkXwJmcwu7hABTyspKI9b', '47bikoA9qfXDT5VVhzsjy9', '5z1A6hdw1MJrmZzDusE4Q6', '08RIk2wKqMNCSzEaZCcHhi', '6ZbHMh1injVPBZTXnDKnYp', '78IYTCUA99H1rwKXjTYSCO', '3ejnp6KkIXiZBWsCzujQ6K', '49KNl5r9iJkG8BYo8VlWxX', '2BeLGTqy54PhX1YT1jTts9', '5VnHdgsAhzWNkd29Rv9ErH', '3TSPcCQfHfv9ga1UTKeBTH', '1L1Hmf2PkeZbiiQbgD93hC', '6An7hnu

In [21]:
len(result)

249

# Create for loop

In [20]:
for i in zip(username,playlist_id):
    print(i)
    get_features_df(i[0],i[1])

NameError: name 'username' is not defined

In [16]:
big_df

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms
0,0.865,0.665,-6.289,0.073,0.0453,0.0,0.224,0.331,111.113,1ofhfV90EnYhEr7Un2fWiv,268827
1,0.801,0.91,-6.566,0.225,0.00239,0.000891,0.0737,0.805,87.277,5g9lS8deSIxItFBmZRC4vN,278387
2,0.644,0.846,-5.443,0.275,0.124,0.0,0.353,0.503,79.182,1qZqFtSINOOmjYH1UE9805,301600
3,0.802,0.787,-6.009,0.206,0.00127,0.000181,0.0763,0.806,90.189,7sKt5Y2zJpYw78iAxndB8y,296907
4,0.872,0.723,-5.968,0.247,0.0293,0.0,0.327,0.843,93.279,4VQNCzfZ3MdHEwwErNXpBo,307773
...,...,...,...,...,...,...,...,...,...,...,...
197,0.76,0.585,-7.307,0.161,0.0102,0.000468,0.308,0.455,91.179,6uWez0py1zc17F7hjp7jIt,311040
198,0.888,0.584,-13.308,0.112,0.0385,0.000003,0.0625,0.762,104.705,5vpDsiVqiZLRwKAzQO708k,334440
199,0.916,0.844,-3.967,0.236,0.0394,0.0,0.0778,0.586,95.19,0Z2J91b2iTGLVTZC4fKgxf,312627
200,0.711,0.693,-8.208,0.41,0.106,0.0,0.0821,0.906,177.978,0LfoVjNyrzkCLzarDAazCh,229305


In [17]:
big_df.to_csv("6383_rows.csv",index=False)

# Loop for 6000 playlist from The sound of Spotify


## TBD - continue in another workbook with the 6000 big_df

In [18]:
for i in result[0]:
    print(i)
    #get_features_df(i[0],'thesoundsofspotify')

6TySxsLwRVYqEh7LZ6fyq8
0OGpuF8rnCzlcjJLsdywny
1BYospE2cLB28m3i84dcTV
6gS3HhOiI17QNojjPuPzqc
5nUyv9wyZKjdUW1rk3smTo
7nHgmv7uyIA1KHj6qTttjH
0lXSqruCjvX1e7hw2KGvKw
41997GIfsUgMnsPLFiuLeS
1aYiM4zLmBuFq0Fg6NQb6a
10FCW9lj0NdeoYI5VVvVtY
6TGOKSobwpEoOtx80edFyV
29qGA62efPsgLp6kblUPSg
67qIuPZRX12l7yMpxBQpxs
2QYND559WzeB2urcTaOG0j
6rZGKd5ndSUnGeJKuBhUej
2XXreCGINlOqm8wEn7WszA
0ShRnc2cyEyxkLbtZC4Yim
5kdP7VQedYdDzhTvfovu0u
4CfyHGpt8qPlLXlstcE2Wh
1Rn01P3YPioy5QRQoo32Lv
0yVmsZK4AtiNMpby2WqLZY
7mBz3DA4Lt7Bsbtr5TxklP
7AqOmLV0JV4a3GpIU1qxbL
5yagbqWY6gDlqB893zjOmO
